In [1]:
import pandas as pd

x = pd.read_csv('/home/m/dev/ai/scaleout/trainingdata/train.csv', encoding='latin-1')
 
x.drop(columns=['query', 'date', '0', 'user', 'index'], inplace=True)


In [3]:
import os

# Calculate the number of parts
num_parts = len(x) // 10000
remainder = len(x) % 10000

# Create the 'x' directory if it doesn't exist
os.makedirs('x', exist_ok=True)

# Split the dataset into parts of 10,000 entries and write to files
for i in range(num_parts):
    part = x[i*10000:(i+1)*10000]
    part.to_csv(f'x/part_{i+1}.csv', index=False)

# Add the remainder as the last part if it exists
if remainder > 0:
    last_part = x[num_parts*10000:]
    last_part.to_csv(f'x/part_{num_parts+1}.csv', index=False)

print(f"Number of parts: {num_parts + (1 if remainder > 0 else 0)}")
for i in range(num_parts + (1 if remainder > 0 else 0)):
    file_path = f'x/part_{i+1}.csv'
    file_size = os.path.getsize(file_path)
    print(f"Part {i+1} size: {file_size} bytes")

,index,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...,...,...
1599994,1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [9]:
import pandas as pd

email = pd.read_csv('/home/m/dev/scaleout/emails.csv', encoding='latin-1')

email.drop(columns=['file'], inplace=True)


In [33]:
import pandas as pd
import multiprocessing as mp
from tqdm import tqdm


def process_email(row):
    extracted_data = {}
    relevant_fields = ['Message-ID', 'Date', 'From', 'To', 'Subject', 'X-From', 'X-To', 'X-cc', 'X-bcc', 'X-Folder', 'X-Origin', 'X-FileName']
    
    # Extract header information
    for line in row['message'].split('\n'):
        for field in relevant_fields:
            if line.startswith(field + ':'):
                extracted_data[field] = line.split(':', 1)[1].strip()
    
    # Extract the message content
    content_start = row['message'].find('\n\n')  # Assuming content starts after a blank line
    if content_start != -1:
        extracted_data['Content'] = row['message'][content_start:].strip()
    else:
        extracted_data['Content'] = ''  # If no content is found
    
    return extracted_data

# Use all available cores
num_cores = mp.cpu_count()

# Create a pool of workers
pool = mp.Pool(num_cores)

# Process emails in parallel
all_extracted_data = list(tqdm(pool.imap(process_email, [row for _, row in email.iterrows()]), total=len(email)))

# Close the pool
pool.close()
pool.join()

# Create a DataFrame from the extracted data
extracted_df = pd.DataFrame(all_extracted_data)

# Display the first few rows of the extracted data
print(extracted_df.head())

# Save the extracted data to a CSV file
extracted_df.to_csv('extracted_emails.csv', index=False)
print("Extracted data saved to 'extracted_emails.csv'")


100%|██████████| 517401/517401 [01:26<00:00, 5975.80it/s]


                                      Message-ID  \
0  <18782981.1075855378110.JavaMail.evans@thyme>   
1  <15464986.1075855378456.JavaMail.evans@thyme>   
2  <24216240.1075855687451.JavaMail.evans@thyme>   
3  <13505866.1075863688222.JavaMail.evans@thyme>   
4  <30922949.1075863688243.JavaMail.evans@thyme>   

                                    Date                     From  \
0  Mon, 14 May 2001 16:39:00 -0700 (PDT)  phillip.allen@enron.com   
1   Fri, 4 May 2001 13:51:00 -0700 (PDT)  phillip.allen@enron.com   
2  Wed, 18 Oct 2000 03:00:00 -0700 (PDT)  phillip.allen@enron.com   
3  Mon, 23 Oct 2000 06:13:00 -0700 (PDT)  phillip.allen@enron.com   
4  Thu, 31 Aug 2000 05:07:00 -0700 (PDT)  phillip.allen@enron.com   

                        To    Subject           X-From  \
0     tim.belden@enron.com             Phillip K Allen   
1  john.lavorato@enron.com        Re:  Phillip K Allen   
2   leah.arsdall@enron.com   Re: test  Phillip K Allen   
3    randall.gay@enron.com             P

In [34]:
extracted = pd.read_csv('extracted_emails.csv')
extracted.drop(columns=['Message-ID', 'From', 'To', 'X-cc', 'X-bcc', 'X-Folder', 'X-Origin', 'X-FileName', 'Date'], inplace=True)
extracted


# Calculate the number of files needed
num_files = len(extracted) // 10000 + (1 if len(extracted) % 10000 > 0 else 0)

# Split the DataFrame and save to separate CSV files
with tqdm(total=num_files, desc="Saving files") as pbar:
    for i in range(num_files):
        start_idx = i * 10000
        end_idx = min((i + 1) * 10000, len(extracted))
        
        # Create a subset of the DataFrame
        subset = extracted.iloc[start_idx:end_idx]
        
        # Save the subset to a CSV file
        filename = f'emails/part_{i}.csv'
        subset.to_csv(filename, index=False)
        
        pbar.update(1)
        pbar.set_postfix({"Last saved": filename})

print(f"Split the extracted data into {num_files} files of 10,000 entries each (or less for the last file).")


Saving files: 100%|██████████| 52/52 [00:12<00:00,  4.32it/s, Last saved=emails/part_51.csv]

Split the extracted data into 52 files of 10,000 entries each (or less for the last file).
